In [1]:
#!pip install tiktoken

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

from dotenv import load_dotenv

from langchain.callbacks import get_openai_callback
import os
import sys
import json
import tiktoken
load_dotenv()


path = os.path.abspath('')
module_path = os.path.join(path,'..\..')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"\\functions")


from sqldatabase_langchain_utils import SQLDatabaseLangchainUtils


### Função para contar tokens

In [3]:
def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def utf8len(s):
    return len(s.encode('utf-8'))

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)


In [4]:
data = {'basic_prompt':{}, 'openai_demonstration_prompt':{}, 'code_representation_prompt':{}}

## Mondial

In [ ]:

SCHEMA = 'mondial_gpt'
json_file_path = f"../../datasets/{SCHEMA}_db_connection.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    db_connection = json.load(json_data, strict=False)

db = SQLDatabaseLangchainUtils(db_connection=db_connection)
exclusao = [
    f"{SCHEMA}_tmdp",
    f"{SCHEMA}_tmdpmap",
    f"{SCHEMA}_tmds",
    f"{SCHEMA}_tmjmap",
    f"{SCHEMA}_tpv",
    f"{SCHEMA}_tmdc",
    f"{SCHEMA}_tmdcmap",
    f"{SCHEMA}_tmdej",
    f"{SCHEMA}_log_action",
    f"{SCHEMA}_log_error",
    f"{SCHEMA}_favorite_item", 
    f"{SCHEMA}_favorite_query",
    f"{SCHEMA}_favorite_tag",
    f"{SCHEMA}_favorite_tag_item",
    f"{SCHEMA}_favorite_visualization",
    f"{SCHEMA}_dashboard",
    f"{SCHEMA}_history"
]

include_tables = [s for s in db.get_table_names() if s.startswith(SCHEMA) and s not in exclusao]
include_tables
db_mondial = SQLDatabaseLangchainUtils(db_connection=db_connection, include_tables=include_tables)

### Basic Prompt

In [6]:
data['basic_prompt']['mondial_size'] = utf8len(db_mondial.get_schema_basic_prompt().strip())
data['basic_prompt']['mondial_tokens_usage'] = num_tokens_from_string(db_mondial.get_schema_basic_prompt().strip())
print(data['basic_prompt']['mondial_tokens_usage'])
print(llm.get_num_tokens(db_mondial.get_schema_basic_prompt().strip()), "tokens")

c:\Users\rogerrsn\Documents\Tecgraf\Danke-General\gpt-llm-to-keywords\experiments\Analises - Text-to-SQL\..\..\functions\sqldatabase_langchain_utils.py:98: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  return self.db._metadata.sorted_tables


1787
1787 tokens


In [7]:
print(db_mondial.get_schema_basic_prompt())

Table continent, columns = [*,name,area]
Table country, columns = [*,name,code,capital,province,area,population]
Table desert, columns = [*,name,area,coordinates]
Table island, columns = [*,name,islands,area,elevation,type,coordinates]
Table lake, columns = [*,name,river,area,elevation,depth,height,type,coordinates]
Table mondial_airport, columns = [*,name,iatacode,country,city,province,island,latitude,longitude,elevation,gmtoffset,meta_repcol]
Table mondial_borders, columns = [*,country1,country2,length,meta_repcol]
Table mondial_city, columns = [*,name,country,province,population,latitude,longitude,elevation,meta_repcol]
Table mondial_cityothername, columns = [*,city,country,province,othername,meta_repcol]
Table mondial_citypops, columns = [*,city,country,province,year,population,meta_repcol]
Table mondial_continent, columns = [*,name,area,meta_repcol]
Table mondial_country, columns = [*,name,code,capital,province,area,population,meta_repcol]
Table mondial_countrypops, columns = [*,c

### OpenAI Demonstration Prompt

In [8]:
data['openai_demonstration_prompt']['mondial_size'] = utf8len(db_mondial.get_schema_openai_prompt().strip())
data['openai_demonstration_prompt']['mondial_tokens_usage'] = num_tokens_from_string(db_mondial.get_schema_openai_prompt().strip())
print(data['openai_demonstration_prompt']['mondial_tokens_usage'])
print(llm.get_num_tokens(db_mondial.get_schema_openai_prompt().strip()), "tokens")

1434
1434 tokens


In [9]:
print(db_mondial.get_schema_openai_prompt())

# continent (name, area)
# country (name, code, capital, province, area, population)
# desert (name, area, coordinates)
# island (name, islands, area, elevation, type, coordinates)
# lake (name, river, area, elevation, depth, height, type, coordinates)
# mondial_airport (name, iatacode, country, city, province, island, latitude, longitude, elevation, gmtoffset, meta_repcol)
# mondial_borders (country1, country2, length, meta_repcol)
# mondial_city (name, country, province, population, latitude, longitude, elevation, meta_repcol)
# mondial_cityothername (city, country, province, othername, meta_repcol)
# mondial_citypops (city, country, province, year, population, meta_repcol)
# mondial_continent (name, area, meta_repcol)
# mondial_country (name, code, capital, province, area, population, meta_repcol)
# mondial_countrypops (country, year, population, meta_repcol)
# mondial_desert (name, area, coordinates, meta_repcol)
# mondial_economy (country, gdp, agriculture, service, industry, infl

### Code Representation Prompt

In [10]:
data['code_representation_prompt']['mondial_size'] = utf8len(db_mondial.get_table_info().strip())
data['code_representation_prompt']['mondial_tokens_usage'] = num_tokens_from_string(db_mondial.get_table_info().strip())
print(data['code_representation_prompt']['mondial_tokens_usage'])
print(llm.get_num_tokens(db_mondial.get_table_info().strip()), "tokens")

7489
7489 tokens


c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\utilities\sql_database.py:307: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


In [11]:
print(db_mondial.get_table_info())


CREATE TABLE airport (
	iatacode VARCHAR(3 CHAR) NOT NULL, 
	name VARCHAR(100 CHAR), 
	country VARCHAR(4 CHAR), 
	city VARCHAR(50 CHAR), 
	province VARCHAR(50 CHAR), 
	island VARCHAR(50 CHAR), 
	latitude NUMBER, 
	longitude NUMBER, 
	elevation NUMBER, 
	gmtoffset NUMBER, 
	CONSTRAINT sys_c00163097 PRIMARY KEY (iatacode), 
	CONSTRAINT airport_city_province_fk FOREIGN KEY(city, country, province) REFERENCES city (name, country, province), 
	CONSTRAINT airport_country_fk FOREIGN KEY(country) REFERENCES country (code), 
	CONSTRAINT airport_island_fk FOREIGN KEY(island) REFERENCES island (name), 
	CONSTRAINT airport_province_country_fk FOREIGN KEY(province, country) REFERENCES province (name, country), 
	CONSTRAINT airplat CHECK ((Latitude >= -90) AND (Latitude <= 90)), 
	CONSTRAINT airplon CHECK ((Longitude >= -180) AND (Longitude <= 180))
)


CREATE TABLE borders (
	country1 VARCHAR(4 CHAR) NOT NULL, 
	country2 VARCHAR(4 CHAR) NOT NULL, 
	length NUMBER, 
	CONSTRAINT borderkey PRIMARY KEY

### Basic Prompt

### Resultados

In [12]:
data

{'basic_prompt': {'mondial_size': 6576, 'mondial_tokens_usage': 1787},
 'openai_demonstration_prompt': {'mondial_size': 5382,
  'mondial_tokens_usage': 1434},
 'code_representation_prompt': {'mondial_size': 29394,
  'mondial_tokens_usage': 7489}}

In [13]:
import pandas as pd

df = pd.DataFrame(data)

In [14]:

df 

,basic_prompt,openai_demonstration_prompt,code_representation_prompt
mondial_size,6576,5382,29394
mondial_tokens_usage,1787,1434,7489


In [15]:
df.index
df.rename(index={'mondial_size': 'Mondial - Size (bytes)', 'mondial_tokens_usage':'Mondial - Tokens Usage'
                 }, inplace=True)

In [16]:
df

,basic_prompt,openai_demonstration_prompt,code_representation_prompt
Mondial - Size (bytes),6576,5382,29394
Mondial - Tokens Usage,1787,1434,7489


In [17]:
filename = 'Análise da Representação dos Prompts.csv'
dir = 'results/'
df.to_csv(dir+filename)